Derive Orbital Elements
<br>
<br>
    - Semi-major axis: The distance from the center of an ellipse to its furthest edge, along the major axis
<br>    
    - Eccentricity: Measure of how elliptical the orbit is around the sun, and is represented by a value between 0 and 1: 
<br>
    Eccentricity of 0: A perfect circle 
<br>
    Eccentricity of 1: A parabola
<br>
<br>
    - Inclination: The angle between the plane of the orbit and the ecliptic plane(Plane of Earth's orbit around the sun)
<br>
<br>
    - Longitude of the ascending node: The angle between the reference direction and the point at which the orbit crosses the ecliptic plane from the south to the north
<br>
<br>
    - Argument of Perihelion: The angle between the ascending node and the point at which the planet is closest to the sun (An argument of periapsis of 0° means that the orbiting body's Periapsis is at the Ascending Node. An argument of periapsis of 90° means that the orbiting body will reach periapsis at its northmost distance from the plane of reference.)

In [32]:
import pandas as pd
import numpy as np

columns = ['Date', 'Position_x', 'Position_y', 'Position_z', 'Velocity_x', 'Velocity_y', 'Velocity_z']
df = pd.DataFrame(columns=columns)

with open('RawData/horizons_results_30_years.csv') as f:
    f.readline()
    for line in f:
        line = line.strip().split(',')

        date = line[0]
        position = np.array(list(map(np.float128, line[1:4])))
        velocity = np.array(list(map(np.float128, line[4:7])))
        new_row = pd.DataFrame([[date, *position, *velocity]], columns=columns)
        df = pd.concat([df, new_row], ignore_index=True)

GM = np.float128(1.32712440018e11) # km^3/s^2

df.head()

/tmp/ipykernel_68061/969097261.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


,Date,Position_x,Position_y,Position_z,Velocity_x,Velocity_y,Velocity_z
0,1992-09-30,-2.353658e+07,-1.056952e+08,-42094.061217,32.644443,-9.032771,-1.945275
1,1992-10-01,-2.069097e+07,-1.064476e+08,-212555.484600,33.233729,-8.366555,-2.001262
2,1992-10-02,-1.779289e+07,-1.071377e+08,-387975.673075,33.854380,-7.588821,-2.059448
3,1992-10-03,-1.484083e+07,-1.077554e+08,-568383.287081,34.478586,-6.689384,-2.116079
4,1992-10-04,-1.183574e+07,-1.082900e+08,-753480.878332,35.076722,-5.665242,-2.167398


In [33]:
position = df[['Position_x', 'Position_y', 'Position_z']].values.astype(np.float128)
velocity = df[['Velocity_x', 'Velocity_y', 'Velocity_z']].values.astype(np.float128)

angular_momentum = np.cross(position, velocity)
vel_cross_h = np.cross(velocity, angular_momentum)

angular_momentum_magnitude = np.sqrt(angular_momentum[:,0]**2 + angular_momentum[:,1]**2 + angular_momentum[:,2]**2)
pos_magnitude = np.sqrt(np.sum(position**2, axis=1, dtype=np.float128))

In [34]:
#Caclulate semi-major axis
pos_x = df['Position_x'].values
pos_y = df['Position_y'].values
pos_z = df['Position_z'].values
vel_x = df['Velocity_x'].values
vel_y = df['Velocity_y'].values
vel_z = df['Velocity_z'].values

r_magnitude = np.sqrt(pos_x**2 + pos_y**2 + pos_z**2)
v_magnitude = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)

semi_major_axis_arr = 1/((2/r_magnitude) - (v_magnitude**2/GM))

semi_major_axis = np.mean(semi_major_axis_arr)

print(f'Mean Semi-major axis: {semi_major_axis} kms')
print("True value of semi-major axis of Venus: 108200000 km")
print("Relative error: ", ((semi_major_axis - 108200000)/108200000)*100, "%")

with open('Results/constants.txt', 'w') as f:
    f.write(f'Mean Semi-major axis: {semi_major_axis} kms\n')

Mean Semi-major axis: 109444168.34983853 kms
True value of semi-major axis of Venus: 108200000 km
Relative error:  1.1498783270226633111 %


In [ ]:
periheleion = np.min(r_magnitude) #Closest to the sun
apoheleion = np.max(r_magnitude)

#convert to AU
periheleion = periheleion
apoheleion = apoheleion

print('Periheleion: ', periheleion)
print('Apoheleion: ', apoheleion)
print("True value of periheleion of Venus: 107471110 kms")
print("True value of apoheleion of Venus: 108937169 kms")
eccentricity = (apoheleion - periheleion)/(apoheleion + periheleion)
print('Eccentricity: ', eccentricity)
print("True value of eccentricity of Venus: 0.0067")
print("Relative error: ", ((eccentricity - 0.0067)/0.0067)*100, "%")

with open('Results/constants.txt', 'a') as f:
    f.write('Periheleion: ' + str(periheleion) + ' kms\n')
    f.write('Apoheleion: ' + str(apoheleion) + ' kms\n')
    f.write('Eccentricity: ' + str(eccentricity) + '\n')

Periheleion:  106777695.57047527538
Apoheleion:  109637482.51564788659
True value of periheleion of Venus: 107471110 kms
True value of apoheleion of Venus: 108937169 kms
Eccentricity:  0.013214354790007146316
True value of eccentricity of Venus: 0.0067
Relative error:  97.22917597025590851 %


In [36]:
#calculate average inclination
inclination = np.arccos(angular_momentum[:,2]/angular_momentum_magnitude)
inclination = np.degrees(inclination)
inclination = np.mean(inclination)
print(f'Mean Inclination: {inclination} degrees')
print("True value of inclination of Venus: 3.39 degrees")
print("Relative error: ", ((inclination - 3.39)/3.39)*100, "%")

with open('Results/constants.txt', 'a') as f:
    f.write(f'Mean Inclination: {inclination} degrees\n')

Mean Inclination: 3.3940019733655222 degrees
True value of inclination of Venus: 3.39 degrees
Relative error:  0.11805231166732356996 %


In [37]:
#Longitude of ascending node
omega = np.arctan2(angular_momentum[:,0], -1*angular_momentum[:,1])
omega = np.degrees(omega)
omega = np.mean(omega)
print(f'Mean Longitude of ascending node: {omega} degrees')
print("True value of Longitude of ascending node of Venus: 76.785 degrees")
print("Relative error: ", abs(((omega - 76.785)/76.785)*100), "%")

with open('Results/constants.txt', 'a') as f:
    f.write(f'Mean Longitude of ascending node: {omega} degrees\n')

Mean Longitude of ascending node: 76.65871657844771 degrees
True value of Longitude of ascending node of Venus: 76.785 degrees
Relative error:  0.16446366028819717857 %


In [38]:
#Calculate Argument of periapsis
node_vector = np.cross([0,0,1], angular_momentum) #direction of the ascending node; Lies in the reference plane; Points from the central body toward the ascending node.
node_vector_magnitude = np.sqrt(node_vector[:,0]**2 + node_vector[:,1]**2 + node_vector[:,2]**2)
node_vector_unit = node_vector/node_vector_magnitude[:,np.newaxis]

e_vector = vel_cross_h/GM - position/pos_magnitude[:,np.newaxis] #eccentricity vector
e_vector_magnitude = np.sqrt(e_vector[:,0]**2 + e_vector[:,1]**2 + e_vector[:,2]**2)
e_vector_unit = e_vector/e_vector_magnitude[:,np.newaxis]

w = np.arccos(np.dot(node_vector_unit, e_vector_unit.T))
w = np.degrees(w)
w = np.mean(w)
print(f'Mean Argument of periapsis: {w} degrees')
print("True value of Argument of periapsis of Venus: 54.78 degrees")
print("Relative error: ", ((w - 54.78)/54.78)*100, "%")

with open('Results/constants.txt', 'a') as f:
    f.write(f'Mean Argument of periapsis: {w} degrees\n')

Mean Argument of periapsis: 88.29658383476765 degrees
True value of Argument of periapsis of Venus: 54.78 degrees
Relative error:  61.18397925295297631 %
